# Query ข้อมูลโรคต่างๆ จาก Reddit#

In [2]:
pip install praw

Note: you may need to restart the kernel to use updated packages.


In [2]:
import praw
import pandas as pd
import datetime as dt

* ข้อมูลเพื่อเข้า API ของ Reddit
    * client_id      = ''
    * client_secret  = '' 
    * user_agent     = '' 
    * username       = '' 
    * password       = ''

In [ ]:
# ใส่ข้อมูลเพื่อเข้า API ของ Reddit
reddit = praw.Reddit(client_id='', 
                     client_secret='', 
                     user_agent='', 
                     username='', 
                     password='')

### สร้าง fuction เพื่อเอาไว้ เปลี่ยนเวลา แบบ Unix time ให้เป็นวันเวลา แบบมาตรฐาน ###
* unix time คือมาตรฐานเวลาที่ถูกนับเริ่มมาตั้งแต่วันที่ 1 มกราคม ค.ศ.1970 โดยถูกนับเพิ่มเรื่อยมาทุกๆวินาที ดังนั้นเวลา unix time ก็คือจำนวนวินาทีที่นับมาตั้งแต่ 1 มกราคม ค.ศ.1970 นั่นเอง และยังคงนับต่อไป
* ดูเวลา Unix time ในปัจจุันได้ที่ : http://service.meewebfree.com/php/unixtime

In [4]:
# function เปลี่ยนแปลง Unix time
def change_date(created):
    return dt.datetime.fromtimestamp(created)

### สร้าง function เพื่อค้นหา กระทู้ ###

In [5]:
def search_reddit(topics, number):  #input topics(type string): หัวข้อกระทู้ที่จะค้นหา and number(integer type ) จำนวนข้อมูลที่ต้องการ
    
    subreddit = reddit.subreddit(topics)  #กำหนดหัวข้อกระทู้
    top_subreddit = subreddit.top(limit=number)   #ดึง top ของกระทู้นั้นๆ และ กำหนด limitที่ต้องการ
    
    
    # สร้าง Data dict เอาไว้เก็บค่าจากกระทู้ที่ค้นหา key : ชื่อ column, value : list
    topics_dict = { "title":[], "score":[], "id":[], "url":[],"comms_num": [], "created": [],"body":[]}
    
    # ใช้ loop for เพื่อทำการดึงข้อมูล ไปเก็บใน Data dict ที่สร้างไว้ข้างบน
    for submission in top_subreddit:
        topics_dict["title"].append(submission.title)
        topics_dict["score"].append(submission.score)
        topics_dict["id"].append(submission.id)
        topics_dict["url"].append(submission.url)
        topics_dict["comms_num"].append(submission.num_comments)
        topics_dict["created"].append(submission.created)
        topics_dict["body"].append(submission.selftext)
    
    
    # นำ Data dict มาสร้างเป็น DataFrame
    topics_data = pd.DataFrame(topics_dict)
    
    
    '''
        ทำการดึงเอา Unix time ในแต่ละ row ของ column created มาแปลงค่าให้เป็น วันเวลา แบบปกติ
    โดย apply จาก function change_date ที่สร้างไว้ด้านบน
    
    '''
    new_time = topics_data.created.apply(change_date)
    
    # ทำการเพิ่ม column timestamp
    topics_data = topics_data.assign(timestamp = new_time)
    
    topics_data = topics_data.drop(['created'], axis=1)
    
    

    #ส่ง Dataframe
    return topics_data

#### ทำการค้นหา hashtag และ export ข้อมูลลงเครื่อง ####

### Covid19 ###

In [65]:
reddit_covid19 = search_reddit("COVID19", 2000)

In [66]:
reddit_covid19.to_excel(r'D:\my_data\reddit_covid19.xlsx', encoding='utf-8-sig', index = False)

In [67]:
reddit_covid19.head()

,title,score,id,url,comms_num,body,timestamp
0,Please consider downloading BOINC or folding@h...,2391,fd29vj,https://www.reddit.com/r/COVID19/comments/fd29...,1047,Hello all.\n\nI believe this has been posted b...,2020-03-04 12:56:00
1,CDC recommends cancelling or postponing all pu...,2184,fjbv0q,https://www.cdc.gov/coronavirus/2019-ncov/comm...,444,,2020-03-16 15:35:33
2,“We were able to ascertain that patients who h...,1998,fkizd0,https://www.mediterranee-infection.com/wp-cont...,442,,2020-03-18 18:33:00
3,Data from SARS outbreak showed that mask weari...,1795,ffy8av,https://www.cochranelibrary.com/cdsr/doi/10.10...,476,,2020-03-10 08:26:30
4,Relationship between the ABO Blood Group and t...,1734,fjzjpc,https://www.medrxiv.org/content/10.1101/2020.0...,407,,2020-03-17 20:05:45


### ebola ###

In [6]:
reddit_ebola = search_reddit("Ebola", 2000)

In [7]:
reddit_ebola.to_excel(r'D:\my_data\reddit_ebola.xlsx', encoding='utf-8-sig', index = False)

In [8]:
reddit_ebola.head()

,title,score,id,url,comms_num,body,timestamp
0,I met 25-year-old Moses Massaquoi today. He's ...,1957,2jg459,http://i.imgur.com/kLhaux5.jpg,129,,2014-10-17 10:43:08
1,"Ebola survivor Christopher, 43, washes blue pa...",1738,2jii6m,http://i.imgur.com/eEhQPJu.jpg,91,,2014-10-18 04:12:32
2,Yesterday I showed you the ebola kit on our IC...,1296,2jeilp,http://imgur.com/a/ZQWR4,493,,2014-10-17 00:05:01
3,"outside an Ebola hospital, guy pulls a mattres...",997,2j7j4s,http://i.imgur.com/19857Nm.jpg,251,,2014-10-15 02:59:30
4,NYC Doctor tests positive for ebola,746,2k5fkp,http://www.nytimes.com/2014/10/24/nyregion/cra...,822,,2014-10-24 15:30:47


In [9]:
reddit_ebola.shape

(1000, 7)

In [10]:
reddit_ebola.to_excel(r'D:\my_data\ebola.xlsx', index = False)

C:\Users\meech\AppData\Local\Continuum\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:915: UserWarning: Ignoring URL 'https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=newssearch&cd=1&cad=rja&uact=8&ved=0CB0QqQIoADAA&url=http%3A%2F%2Ffox13now.com%2F2014%2F10%2F03%2Fprimary-childrens-hospital-confirms-patient-does-not-have-ebola%2F&ei=hD0vVOOcNLTGsQTZtICoDw&usg=AFQjCNG9pBIdes91lXiHDnz7PEKijOzitA&sig2=vDgd7Gd6C1oYcWodTC9a5g' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Users\meech\AppData\Local\Continuum\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:915: UserWarning: Ignoring URL 'http://www.reddit.com/r/news/comments/2jfwyp/another_ebola_case_inova_hospital_in_northern?sort=new

Apparently%20this%20guy%20was%20an%20inmate%20and%20had%20come%20from%20W%20Africa%20within%20the%20past%2021%20days.%20Let's%20see%20if%20reddit%20is%20correct%20and%20beats%20the%20msm%20to%20the%20punch%20again.' with link or lo